In [31]:
! pip install acryl-datahub[great-expectations]


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.3/157.3 kB 169.6 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 488.7 kB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.6/151.6 kB 1.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.9 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of ipython to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of jupyter-core to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of jupyter-core to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide 

In [32]:
import great_expectations as gx
from great_expectations.core.batch import BatchRequest, RuntimeBatchRequest
from ruamel import yaml

In [33]:
context = gx.get_context()

In [34]:
config = """
name: trino_datasource
class_name: Datasource
execution_engine:
  class_name: SqlAlchemyExecutionEngine
  connection_string: trino://trino@trino:8080/delta/my_schema
data_connectors:
   default_runtime_data_connector_name:
       class_name: RuntimeDataConnector
       batch_identifiers:
           - default_identifier_name
   default_inferred_data_connector_name:
       class_name: InferredAssetSqlDataConnector
       include_schema_name: true
"""

In [35]:
context.test_yaml_config(config)

Attempting to instantiate class from config...
	Instantiating as a Datasource, since class_name is Datasource
	Successfully instantiated Datasource


ExecutionEngine class name: SqlAlchemyExecutionEngine
Data Connectors:
	default_inferred_data_connector_name : InferredAssetSqlDataConnector

	Available data_asset_names (2 of 2):
		my_schema.appl_stock_delta_table (1 of 1): [{}]
		my_schema.appl_stock_delta_table_version_2 (1 of 1): [{}]

	Unmatched data_references (0 of 0):[]

	default_runtime_data_connector_name:RuntimeDataConnector

	Available data_asset_names (0 of 0):
		Note : RuntimeDataConnector will not have data_asset_names until they are passed in through RuntimeBatchRequest

	Unmatched data_references (0 of 0): []



In [36]:
context.add_datasource(**yaml.load(config))

In [37]:
trino = context.get_datasource("trino_datasource")

trino.get_available_data_asset_names()

{'default_runtime_data_connector_name': [],
 'default_inferred_data_connector_name': ['my_schema.appl_stock_delta_table',
  'my_schema.appl_stock_delta_table_version_2']}

In [38]:
batch_request = BatchRequest(
    datasource_name="trino_datasource",
    data_connector_name="default_inferred_data_connector_name",
    data_asset_name="my_schema.appl_stock_delta_table",  # this is the name of the table you want to retrieve
)
context.add_expectation_suite(
    expectation_suite_name="test_suite"
)
validator = context.get_validator(
    batch_request=batch_request, expectation_suite_name="test_suite"
)

In [39]:
validator.expect_column_to_exist("close")

Calculating Metrics:   0%|          | 0/2 [00:00<?, ?it/s]

{
  "success": true,
  "result": {},
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [40]:
validator.head()

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

,date,open,high,low,close,volume,adjclose
0,2010-01-04,213.429998,214.499996,212.380001,214.009998,123432400,27.727039
1,2010-01-05,214.599998,215.589994,213.249994,214.379993,150476200,27.774976
2,2010-01-06,214.379993,215.230000,210.750004,210.969995,138040000,27.333178
3,2010-01-07,211.750000,212.000006,209.050005,210.580000,119282800,27.282650
4,2010-01-08,210.299994,212.000006,209.060005,211.980005,111902700,27.464034


In [41]:
validator.expect_column_min_to_be_between("close",0)

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": 90.279999
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

## Save Expectations to Expectation Suite

In [42]:
validator.save_expectation_suite()

In [43]:
validator.get_expectation_suite()

{
  "expectation_suite_name": "test_suite",
  "ge_cloud_id": null,
  "expectations": [
    {
      "expectation_type": "expect_column_to_exist",
      "kwargs": {
        "column": "close"
      },
      "meta": {}
    },
    {
      "expectation_type": "expect_column_min_to_be_between",
      "kwargs": {
        "column": "close",
        "min_value": 0
      },
      "meta": {}
    }
  ],
  "data_asset_type": null,
  "meta": {
    "great_expectations_version": "0.18.18"
  }
}

In [44]:
context.get_available_data_asset_names()

{'trino_datasource': {'default_runtime_data_connector_name': [],
  'default_inferred_data_connector_name': ['my_schema.appl_stock_delta_table',
   'my_schema.appl_stock_delta_table_version_2']}}

## Create A Checkpoint from Our DataSource and Expectation Suite

In [45]:
checkpoint_config = """
name: test_checkpoint 
config_version: 1
class_name: SimpleCheckpoint
validations:
  - batch_request:
      datasource_name: trino_datasource  # Update this value.
      data_connector_name: default_inferred_data_connector_name  # Update this value.
      data_asset_name: my_schema.appl_stock_delta_table  # Update this value.
      data_connector_query:
        index: -1
    expectation_suite_name: test_suite  # Update this value.
"""

In [46]:
## Add Datahub Integration if Using Datahub (note indentation: action_list is a key inside of validations
checkpoint_config += """
    action_list:
      - name: datahub_action
        action:
          module_name: datahub.integrations.great_expectations.action
          class_name: DataHubValidationAction
          server_url: http://datahub-gms:8080 #DataHub server url
"""

In [47]:
print(checkpoint_config)


name: test_checkpoint 
config_version: 1
class_name: SimpleCheckpoint
validations:
  - batch_request:
      datasource_name: trino_datasource  # Update this value.
      data_connector_name: default_inferred_data_connector_name  # Update this value.
      data_asset_name: my_schema.appl_stock_delta_table  # Update this value.
      data_connector_query:
        index: -1
    expectation_suite_name: test_suite  # Update this value.

    action_list:
      - name: datahub_action
        action:
          module_name: datahub.integrations.great_expectations.action
          class_name: DataHubValidationAction
          server_url: http://datahub-gms:8080 #DataHub server url



In [48]:
context.test_yaml_config(yaml_config=checkpoint_config)

Attempting to instantiate class from config...
	Instantiating as a SimpleCheckpoint, since class_name is SimpleCheckpoint
	Successfully instantiated SimpleCheckpoint


Checkpoint class name: SimpleCheckpoint


{
  "action_list": [
    {
      "name": "store_validation_result",
      "action": {
        "class_name": "StoreValidationResultAction"
      }
    },
    {
      "name": "store_evaluation_params",
      "action": {
        "class_name": "StoreEvaluationParametersAction"
      }
    },
    {
      "name": "update_data_docs",
      "action": {
        "class_name": "UpdateDataDocsAction"
      }
    }
  ],
  "batch_request": {},
  "class_name": "SimpleCheckpoint",
  "config_version": 1.0,
  "evaluation_parameters": {},
  "module_name": "great_expectations.checkpoint",
  "name": "test_checkpoint",
  "profilers": [],
  "runtime_configuration": {},
  "validations": [
    {
      "action_list": [
        {
          "name": "datahub_action",
          "action": {
            "module_name": "datahub.integrations.great_expectations.action",
            "class_name": "DataHubValidationAction",
            "server_url": "http://datahub-gms:8080"
          }
        }
      ],
      "batch_req

In [49]:
context.add_checkpoint(**yaml.load(checkpoint_config))

{
  "action_list": [
    {
      "name": "store_validation_result",
      "action": {
        "class_name": "StoreValidationResultAction"
      }
    },
    {
      "name": "store_evaluation_params",
      "action": {
        "class_name": "StoreEvaluationParametersAction"
      }
    },
    {
      "name": "update_data_docs",
      "action": {
        "class_name": "UpdateDataDocsAction"
      }
    }
  ],
  "batch_request": {},
  "class_name": "SimpleCheckpoint",
  "config_version": 1.0,
  "evaluation_parameters": {},
  "module_name": "great_expectations.checkpoint",
  "name": "test_checkpoint",
  "profilers": [],
  "runtime_configuration": {},
  "validations": [
    {
      "action_list": [
        {
          "name": "datahub_action",
          "action": {
            "module_name": "datahub.integrations.great_expectations.action",
            "class_name": "DataHubValidationAction",
            "server_url": "http://datahub-gms:8080"
          }
        }
      ],
      "batch_req

In [50]:
context.run_checkpoint("test_checkpoint")

PluginModuleNotFoundError: No module named `datahub.integrations.great_expectations.action` could be found in your plugins directory.
    - Please verify your plugins directory is configured correctly.
    - Please verify you have a module named `datahub.integrations.great_expectations.action` in your plugins directory.
